<a href="https://colab.research.google.com/github/bayashi-cl/statistical-learning/blob/main/note/06_LinerModelSelectionAndRegularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 線形モデル選択と正則化

この章では最小二乗法以外の線形モデルについて考える。

* **部分集合選択** モデルに含まれる予測変数の集合のうち、最も「良い」部分集合を特定し、その部分集合に対して最小二乗法を適用する。
* **縮小推定** 予測変数をすべて使って推定をするが、何らかの操作により係数の推定値を0やそれに近い値にする。
* **次元削減** $p$個の予測変数を$M(<p)$次元部分空間に射影して最小二乗法を適用する。

これらのモデルは、最小二乗法に対して次のような利点がある。

* **予測精度** 最小二乗法はデータ数が予測変数の数よりも十分に大きければ十分な性能を発揮するが、そうでない場合は過学習などの問題が発生して分散が大きくなる。縮小推定では回帰係数に制約を課すことで予測精度を向上させる。
* **モデルの解釈可能性** モデルの中に実際には応答変数と関係のない予測変数が含まれていると、モデルが必要以上に複雑になってしまう。このような無意味な変数を取り除くことでモデルの解釈が容易になる。

## 6.1 部分集合選択

### 補足 計算量の評価について

アルゴリズムのコスト（計算回数）のことを（時間）計算量と呼ぶ。計算量を評価する際に、例えば入力サイズが$n$のアルゴリズムの計算コストが$3n^2 + 2n + 5$である場合、注目するべきは$n^2$の部分であり、それ以外の係数や1次より小さい項は$n$が増加した場合の計算量の増加度合いにはあまり関与しない。

数列の**オーダー**の記法を計算量でも用いる。

#### $\Theta(O,o,\Omega , \omega)$記法

2つの計算量の増加度合いが同じ（くらい）であることを言いたい。

$f(n)$が$g(n)$に対して次の条件を満たすとき、$f(n)$のオーダーが$g(n)$であるといい、$f(n) \in \Theta (g(n))$ と表記する。

> ある定数 $n_0$​ に対し、ある定数 $c_L, c_U > 0$ が存在し、$n_0$ 以上のすべての $n$ について次の式が成り立つ：
> $$0 \le c_L \cdot g(n) \le f(n) \le c_U \cdot g(n)$$

また、上の不等式の$c_L$側が成り立つとき、$f(n) \in \Omega (g(n))$、$c_U$側が成り立つとき、$f(n) \in O (g(n))$と表記する。

更に、大小関係が真に成り立つ（$\le$ではなく$\lt$）場合は,
それぞれ$f(n) \in \omega (g(n))$、$f(n) \in o (g(n))$と表記する。

#### 参考文献

* アルゴリズムイントロダクション 第3版 総合版(p36-)（該当部分は[Amazon](https://www.amazon.co.jp/dp/B078WPYHGN)で試し読みができます。）



### 6.1.1 最良部分集合選択

すべての予測変数の組み合わせを評価する手法。以下のアルゴリズムで行われる。

1. $M_0$を予測変数を持たない**ヌルモデル**とする。ヌルモデルは予測値として標本平均を返す。
1. $k = 1, 2, \ldots , p$について以下の手順を行う。
    1. $k$個の予測変数を持つ$\binom{p}{k}$個のモデルに対してRSSや$R^2$を計算する。
    1. 最良のモデルを$M_k$とする。
1. $M_0, \ldots , M_p$から最良のモデルをBICや修正$R^2$などで選ぶ。

step3でのモデル選択の際にはRSSや$R^2$が特徴の数に対して単調減少/増加することに注意する。RSSが小さい・$R^2$が大きいことは単に訓練データに対して誤差が小さいことを示しているということであり、重要なのはテストデータに対する誤差である。

最良部分集合選択は単純であり、得られるモデルも厳密に最良であるといえるが、計算量のオーダーがモデル評価の計算量を$\Theta(V)$として$\Theta(2^pV)$であり、$p \ge 40$の場合には現実的な時間で解くことが難しくなる。

### 6.1.2 ステップワイズ法

モデルを限定することで効率的に探索を行う。

#### 変数増加法

当てはめを最も良くする予測変数を順にモデルに追加していく手法。以下のアルゴリズムで行われる。

1. ヌルモデルを$M_0$とする。
1. $k = 1, 2, \ldots , p$について以下の手順を行う。
    1. $M_k$に含まれない予測変数のうち1つを加えた$(p-k)$個のモデルを考える。
    1. 最良のモデルを$M_{k+1}$とする。
1. $M_0, \ldots , M_p$から最良のモデルをBICや修正$R^2$などで選ぶ。 

変数増加法は$p>n$の場合にも適用することができる。

#### 変数減少法

不要な予測変数を順に削除していく手法。以下のアルゴリズムで行われる。

1. すべての予測変数を含むモデルをを$M_p$とする。
1. $k = p, p-1, \ldots , 1$について以下の手順を行う。
    1. $M_k$に含まれる予測変数のうち1つを除いた$k$個のモデルを考える。
    1. 最良のモデルを$M_{k-1}$とする。
1. $M_0, \ldots , M_p$から最良のモデルをBICや修正$R^2$などで選ぶ。 


#### 変数増減法

増加法と減少法を混ぜたもの。変数増加法と同様に予測変数を追加していくが、その際にモデルの当てはめに貢献しない予測変数を取り除く。

これらの手法の計算量は$\Theta(p^2V)$であり最良部分集合選択よりも優れているが、最良部分集合選択と同一のモデルが選択されるとは限らない。

### 6.1.2 最適モデルの選択

RSSや$R^2$は予測変数の数に依存し、すべての予測変数を含むモデルで最小/最大となるために、予測変数の数の異なるモデルを比較するのに適していない。また、訓練データの誤差の影響を受けているため、テストデータに対する誤差を小さくするためには何らかの修正を加える必要がある。

よく用いられるのは以下の2つの方法である。

1. 過学習によるバイアスを考慮して訓練誤差を修正することで間接的にテスト誤差を推定する。
1. ホールドアウト検証や交差検証法により直接的にテスト誤差を推定する。

#### $C_p$, AIC, BIC

$C_p$, AIC, BICはMSE($:=\textrm{RSS}/n$)に罰則項を追加したものである。それぞれ、

$$C_p = \frac{1}{n}(\textrm{RSS} + 2d\hat{\sigma}^2)$$
$$\textrm{AIC} = \frac{1}{n\hat{\sigma}^2}(\textrm{RSS} + 2d\hat{\sigma}^2)$$
$$\textrm{BIC} = \frac{1}{n\hat{\sigma}^2}(\textrm{RSS} + \log{(n)}d\hat{\sigma}^2)$$

となる（定数項は省略されている）。

ここで、$\hat{\sigma}^2$は応答変数の測定値に対する誤差の分散の推定値であり、これが不偏であれば$C_p$はテストデータのMSEとして不偏であることが言える。すなわち、テストデータでの誤差が小さいモデルは$C_p$も小さくなるため、これが最小のモデルを選べば良い。

$C_p$とAICは比例関係にあるが、これらに対してBICは予測変数が多いモデルに対してより重い罰則を与える。

#### 自由度調整済み$R^2$

自由度調整済み$R^2$は次の式で表される。

$$自由度調整済み R^2 = 1 - \frac{\textrm{RSS}/(n-d-1)}{\textrm{TSS}/(n-1)}$$

モデルに対してRSSをほとんど改善しない予測変数を追加すると、$\textrm{RSS}/(n-d-1)$が増加し、自由度調整済み$R^2$は減少する。

#### ホールドアウト検証・交差検証

ホールドアウト検証や交差検証を使って直接テスト誤差を推定することもできる。この手法はテスト誤差の直接の推定である点や、真のモデルについての過程がAICなどより少ないという点で優れている。従来は$p$や$n$が大きい場合に交差検証の計算量が大きくなるという問題があったが、コンピュータの計算速度の進歩により改善されている。

テスト誤差の推定値は予測変数の個数を変えたとしてもほとんど変化しない場合がある。そのような場合には推定値の標準誤差を計算し、推定値の最小値から1標準誤差以内のモデルのうち最も予測変数の数が少ないものを選択する。これは**1標準誤差ルール**と呼ばれる。

## 考察

### 最良部分集合選択

* 0-1ナップザック問題に似ている
* 似たモデルに対して何回も評価をしていて無駄が大きい？
* 動的計画法で厳密解の計算量を落とせそうな気がする。
* 時間・空間ともに$\Theta(NB\log{p})$ (Bはバケット数)
* DPやるには$R^2$に線形性っぽさが必要？